In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from training_audio_model_ipu import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

data[:3]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


True


,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.54,4.72,0.18,tu,False,False,False,transcr\AAOR
1,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.72,4.84,0.12,as,False,False,True,transcr\AAOR
2,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,5.14,5.44,0.30,mh,True,False,False,transcr\AAOR


In [2]:

df = pd.DataFrame(data)
from utils_ipu import *
y=create_y(df)
print(y.count(1))

In [3]:

audio_files_path = 'paco-cheese/audio/2_channels/'


audio_segments = extract_audio_segments(data,audio_files_path)
audio_segments

 ...]

In [4]:
print(len(audio_segments))
print(len(y))


36451
36451


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier



X = np.array([extract_features(segment) for segment in audio_segments])


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1280
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1120
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=18

In [12]:
dataset = AudioDataset(X, y)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Fixez le seed pour la reproductibilité
seed = 42

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Créez les objets AudioDataset pour l'entraînement et le test
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

# Créez les DataLoaders pour ces ensembles de données
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(len(X))
print(len(y_test))

36451
7291


In [7]:
import copy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
patience=5
best_val=0
best_model=None
model = AudioCNN().to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Paramètres d'entraînement
num_epochs = 10
class_weights = torch.tensor([1.0, 2], device=device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

#entraînement
for epoch in range(num_epochs):

    model.train()
    
    for inputs, labels in train_loader:
    

   
        inputs, labels = inputs.to(device), labels.to(device)
       

        optimizer.zero_grad()


        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        loss.backward()
        optimizer.step()

    # Évaluation
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
          
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    all_preds_audio,all_labels=predition_model_audio(model,test_loader,device,proba=False)
    f1 = f1_score(all_labels, all_preds_audio)
    if f1>best_val:
        best_val=f1
        best_model=copy.deepcopy(model)
        patience=5
    else:
        patience-=1
    if patience==0:
        break
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')
    torch.save(model, 'modele/model_audio')

Using device: cuda
Epoch [1/10], Loss: 0.6102, Accuracy: 72.06%
Epoch [2/10], Loss: 0.6503, Accuracy: 72.09%
Epoch [3/10], Loss: 0.5313, Accuracy: 73.87%
Epoch [4/10], Loss: 0.7645, Accuracy: 67.86%
Epoch [5/10], Loss: 0.6222, Accuracy: 72.51%
Epoch [6/10], Loss: 0.6382, Accuracy: 73.06%
Epoch [7/10], Loss: 0.5335, Accuracy: 71.83%
Epoch [8/10], Loss: 0.5200, Accuracy: 71.03%


In [8]:

#affiche le nombre de y dans y : 
print(y.count(0))
print(y.count(1))
print(y.count(0)/len(y))

26943
9508
0.7391566760857041


In [9]:
all_preds_audio,all_labels=predition_model_audio(best_model,test_loader,device,proba=False)
f1 = f1_score(all_labels, all_preds_audio)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_audio)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')

Test F1 Score: 0.38228315317690487
Confusion Matrix:
[[4223 1233]
 [1110  725]]
Nombre d'éléments de classe 0 détectés : 4223 sur 5456
Nombre d'éléments de classe 1 détectés : 725 sur 1835
